# Rescaling of LCs (Biagio)

The following warning has been ignored, keep in mind when inserting this code in the grblc package

C:\Users\biagi\AppData\Local\Temp\ipykernel_2268\907779747.py:51: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
import os
from grblc.fitting import OutlierPlot
import plotly.offline
import plotly.io as pio
import glob

#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

# ignores FutureWarning for pandas
import warnings
warnings.filterwarnings("ignore")

# 08-12-2022, Last condition - going beyond the most numerous filter
### The duplicates and boundaries conditions are already inserted here

In [ ]:
path = r'C:/Users/biagi/Desktop/LCs to inspect deltat/CLEAN_GRBs_MAGNITUDES_December2022/Debugging/*.txt'

filename = glob.glob(path)


for i in filename:
    try:
        light = pd.read_csv(i, sep=r"\t+\s+", engine="python", header=None, index_col=0).sort_values(by=['Time(s)'])
        light=light[light['MagErr'] != 0]
        if light.empty:
            print(i+" has only limiting magnitudes, thus it will be skipped")
            continue
        color=(light[3].values).to_list()
        print(color)

        filterslist=[[x,color.count(x)] for x in set(color)]
        freq=[]
        for k in color:
            for j in filterslist:
                if j[0]==k:
                    freq.append(str(k)+' ('+str(j[1])+')')

        fig = px.scatter(
                    data_frame=light,
                    x=np.log10(light[0].values),
                    y=-light[1].values,
                    error_y= light[2].values,
                    color=color,
                    hover=light[7].values,
                    title=i,
                )
        
        fig.update_layout(
                    xaxis_title='log Time (s)',
                    yaxis_title='Mag'
                )
        fig.show()
        #fig.write_image(i+".pdf")
        
        if len(freq)==1:
            print(i+" has only one data point along all the LC, thus no rescaling is needed")
            continue
        
    except:
        light = pd.read_csv(i, sep=r"\t+|\s+", engine="python", header=0).sort_values(by=['Time(s)'])#, index_col=0)
        light=light[light['MagErr'] != 0]
        if light.empty:
            print(i+" has only limiting magnitudes, thus it will be skipped")
            continue       
        color=(light['Filter']).to_list()
        filterslist=[[x,color.count(x)] for x in set(color)]
        freq=[]
        for k in color:
            for j in filterslist:
                if j[0]==k:
                    freq.append(str(k)+' ('+str(j[1])+')')
        
        fig = px.scatter(
            data_frame=light,
            x=np.log10(light['Time(s)'].values),
            y=-light['Mag'].values,
            error_y= light['MagErr'].values,
            color=freq,
            hover_data=['Source'],
            title=i,
        )
        
        fig.update_layout(
            xaxis_title='log Time (s)',
            yaxis_title='Mag'
        )
        
        fig.show()
        #fig.write_html(i+".html")
        
        if len(freq)==1:
            print(i+" has only one data point along all the LC, thus no rescaling is needed")
            continue        
        
    light = pd.read_csv(i, sep=r"\t+|\s+", engine="python", header=0).sort_values(by=['Time(s)'])#, index_col=0)
    light=light[light['MagErr'] != 0]
    if light.empty:
        print(i+" has only limiting magnitudes, thus it will be skipped")
        continue
    x=np.log10(light['Time(s)'].values)
    y=light['Mag'].values
    error_y= light['MagErr'].values
    color=(light['Filter']).to_list()
    filterslist=[[x,color.count(x)] for x in set(color)]
    freq=[]
    freq1=[]
    freqfinal=[]
    for k in color:
        for j in filterslist:
            if j[0]==k:
                freq.append(str(k)+' ('+str(j[1])+')')
                freq1.append([k,j[1]])
                freqfinal.append(j[1])
    
    freq_set = set(tuple(x) for x in freq1)
    occurrences = [list(x) for x in freq_set]
    sortedoccur=sorted(occurrences,key=lambda x: int(x[1]),reverse=True)
    
    
    
    # Identifying the most numerous filter and the second most numerous in the GRB 
    mostcommonfilter=sortedoccur[0][0]
    
    if len(sortedoccur)==1:
        print(i+" has only one filter along all the LC, thus no rescaling is needed")
        continue
        
    secondcommonfilter=sortedoccur[1][0]
    
    # FIRST RIDE; WE HERE RESCALE ACCORDING TO THE MOST NUMEROUS FILTER

    # We here create the sublist of data points that have the most common filter
    
    mostcommonlight=light.loc[light['Filter'] == mostcommonfilter].to_dict('subset')
    mostcommonx=np.log10(mostcommonlight['Time(s)'].values)
    mostcommony=mostcommonlight['Mag'].values
    mostcommonerror_y= mostcommonlight['MagErr'].values
    
    # Second step: find the minimum time difference between the most common and the next filter in order of occurrences
    
    # Here we start to create a list that contains the rescaling factors for every filter after the most common
    # The structure is (filtername, number of points, list of rescaling factors (up to 6))
    scalingfactorslist=[[sortedoccur[0][0],sortedoccur[0][1],[0]]]
    
    # The following for loop makes j go through all the other filters (after the most common)
    
    
    # Last step: rescaling all the points according to the rescaling factors
    
    # Defining the overlap function: this function tells you if two ranges of real numbers overlap
    # So, if we consider the ranges (logflux-err,logflux+err) we can compare if two error bars overlap
    
    def overlap(start1, end1, start2, end2):
        #how much does the range (start1, end1) overlap with (start2, end2)
        return max(max((end2-start1), 0) - max((end2-end1), 0) - max((start2-start1), 0), 0)
    
    
    # Here we re-import the GRB data (A RE-IMPORTATION IS HEAVY IN CODING AND NOT NECESSARY, THIS CAN BE IMPROVED)
    lightresc = pd.read_csv(i, sep=r"\t+|\s+", engine="python", header=0).sort_values(by=['Time(s)'])#, index_col=0)
    lightresc=lightresc[lightresc['MagErr'] != 0]
    if lightresc.empty:
        print(i+" has only limiting magnitudes, thus it will be skipped")
        continue  
    rescx=np.log10(lightresc['Time(s)'].values)
    rescy=lightresc['Mag'].values
    resccolor=lightresc['Filter'].values
    rescerror_y= lightresc['MagErr'].values
    reschover_data=lightresc['Source'].values
    
    # 1st part of LC: 0<logt<2 => logdeltat<=0.35
    logdtprecision=0.35
    lightresc1=lightresc.loc[(np.log10(lightresc['Time(s)']) <= 2) & (np.log10(lightresc['Time(s)']) > 0)].to_dict('lightresc1')
    rescx1=np.log10(lightresc1['Time(s)'])
    rescy1=lightresc1['Mag']
    resccolor1=lightresc1['Filter']
    rescerror_y1= lightresc1['MagErr'] 
    reschover_data1=lightresc1['Source']    
    
    mostcommonlight1=light.loc[(light['Filter'] == mostcommonfilter) & (np.log10(lightresc['Time(s)']) <= 2) & (np.log10(lightresc['Time(s)']) > 0)].to_dict('list')
    
    #print(mostcommonlight1)
    
    mostcommonx1=np.log10(mostcommonlight1['Time(s)'])
    mostcommony1=mostcommonlight1['Mag']
    mostcommonerror_y1= mostcommonlight1['MagErr']
    
    secondcommonlight1=light.loc[(light['Filter'] == secondcommonfilter) & (np.log10(lightresc['Time(s)']) <= 2) & (np.log10(lightresc['Time(s)']) > 0)].to_dict('list')
        

    for j in range(1,len(sortedoccur)):
        scalingfactorslist.append([sortedoccur[j][0],sortedoccur[j][1],[]])
                 

    for j in range(1,len(sortedoccur)):
        
        sublight=light.loc[(lightresc['Filter'] == sortedoccur[j][0]) 
                                     & (np.log10(lightresc['Time(s)']) <= 2) & (np.log10(lightresc['Time(s)']) > 0)].to_dict('subset')
        subx=np.log10(sublight['Time(s)'].values)
        suby=sublight['Mag'].values
        suberror_y=sublight['MagErr'].values
        
        
        indexeslist = [[p1,p2] for p1 in range(len(mostcommonx1)) for p2 in range(len(subx))]
        #print(indexeslist)
    
        timediff = [np.abs(mostcommonx1[p1]-subx[p2]) for p1 in range(len(mostcommonx1)) for p2 in range(len(subx))]
        
        if len(timediff)==0:
            continue
        
        else:
            m = min(timediff)
            minpos = timediff.index(m)
    
            # We here define the re-scaling constant between the most common band and the j-th band (in order of occurencies)
            scalingfactor1=mostcommony1[indexeslist[minpos][0]]-suby[indexeslist[minpos][1]]
            scalingfactorslist[j][2].append(scalingfactor1)
        

    # 2nd part of LC: 2<logt<3 => logdeltat<=0.3
    logdtprecision=0.3
    lightresc2=lightresc.loc[(np.log10(lightresc['Time(s)']) <= 3) & (np.log10(lightresc['Time(s)']) > 2)].to_dict('lightresc1')
    rescx2=np.log10(lightresc2['Time(s)'])
    rescy2=lightresc2['Mag']
    resccolor2=lightresc2['Filter']
    rescerror_y2= lightresc2['MagErr']
    reschover_data2=lightresc2['Source']    
    
    mostcommonlight2=light.loc[(light['Filter'] == mostcommonfilter) & (np.log10(lightresc['Time(s)']) <= 3) & (np.log10(lightresc['Time(s)']) > 2)].to_dict('list')
    
    #print(mostcommonlight1)
    
    mostcommonx2=np.log10(mostcommonlight2['Time(s)'])
    mostcommony2=mostcommonlight2['Mag']
    mostcommonerror_y2= mostcommonlight2['MagErr']
    
    
    for j in range(1,len(sortedoccur)):
        
        sublight=light.loc[(lightresc['Filter'] == sortedoccur[j][0]) 
                                     & (np.log10(lightresc['Time(s)']) <= 3) & (np.log10(lightresc['Time(s)']) > 2)].to_dict('subset')
        subx=np.log10(sublight['Time(s)'].values)
        suby=sublight['Mag'].values
        suberror_y=sublight['MagErr'].values
        
        #scalingfactorslist.append([sortedoccur[j][0],sortedoccur[j][1],[]])
        
        indexeslist = [[p1,p2] for p1 in range(len(mostcommonx2)) for p2 in range(len(subx))]
        #print(indexeslist)
    
        timediff = [np.abs(mostcommonx2[p1]-subx[p2]) for p1 in range(len(mostcommonx2)) for p2 in range(len(subx))]
        
        if len(timediff)==0:
            continue
        
        
        else:
            m = min(timediff)
            minpos = timediff.index(m)
    
            # We here define the re-scaling constant between the most common band and the j-th band (in order of occurencies)
            scalingfactor2=mostcommony2[indexeslist[minpos][0]]-suby[indexeslist[minpos][1]]
            scalingfactorslist[j][2].append(scalingfactor2)
        

    # 3rd part of LC: 3<logt<4 => logdeltat<=0.25
    logdtprecision=0.25
    lightresc3=lightresc.loc[(np.log10(lightresc['Time(s)']) <= 4) & (np.log10(lightresc['Time(s)']) > 3)].to_dict('lightresc1')
    rescx3=np.log10(lightresc3['Time(s)'])
    rescy3=lightresc3['Mag']
    resccolor3=lightresc3['Filter']
    rescerror_y3= lightresc3['MagErr']
    reschover_data3=lightresc3['Source']    
    
    mostcommonlight3=light.loc[(light['Filter'] == mostcommonfilter) & (np.log10(lightresc['Time(s)']) <= 4) & (np.log10(lightresc['Time(s)']) > 3)].to_dict('list')
    
    #print(mostcommonlight1)
    
    mostcommonx3=np.log10(mostcommonlight3['Time(s)'])
    mostcommony3=mostcommonlight3['Mag']
    mostcommonerror_y3= mostcommonlight3['MagErr']
    
    
    
    for j in range(1,len(sortedoccur)):
        
        sublight=light.loc[(light['Filter'] == sortedoccur[j][0]) 
                                     & (np.log10(lightresc['Time(s)']) <= 4) & (np.log10(lightresc['Time(s)']) > 3)].to_dict('subset')
        subx=np.log10(sublight['Time(s)'].values)
        suby=sublight['Mag'].values
        suberror_y=sublight['MagErr'].values
        
        #scalingfactorslist.append([sortedoccur[j][0],sortedoccur[j][1],[]])
        
        indexeslist = [[p1,p2] for p1 in range(len(mostcommonx3)) for p2 in range(len(subx))]
        #print(indexeslist)
    
        timediff = [np.abs(mostcommonx3[p1]-subx[p2]) for p1 in range(len(mostcommonx3)) for p2 in range(len(subx))]
        
        if len(timediff)==0:
            continue
        
        else:
            m = min(timediff)
            minpos = timediff.index(m)
    
            # We here define the re-scaling constant between the most common band and the j-th band (in order of occurencies)
            scalingfactor3=mostcommony3[indexeslist[minpos][0]]-suby[indexeslist[minpos][1]]
            scalingfactorslist[j][2].append(scalingfactor3)
                 
                
    # 4th part of LC: 4<logt<5 => logdeltat<=0.2
    logdtprecision=0.2
    lightresc4=lightresc.loc[(np.log10(lightresc['Time(s)']) <= 5) & (np.log10(lightresc['Time(s)']) > 4)].to_dict('lightresc1')
    rescx4=np.log10(lightresc4['Time(s)'])
    rescy4=lightresc4['Mag']
    resccolor4=lightresc4['Filter']
    rescerror_y4= lightresc4['MagErr']
    reschover_data4=lightresc4['Source']    
    
    mostcommonlight4=light.loc[(light['Filter'] == mostcommonfilter) & (np.log10(lightresc['Time(s)']) <= 5) & (np.log10(lightresc['Time(s)']) > 4)].to_dict('list')
    
    #print(mostcommonlight1)
    
    mostcommonx4=np.log10(mostcommonlight4['Time(s)'])
    mostcommony4=mostcommonlight4['Mag']
    mostcommonerror_y4= mostcommonlight4['MagErr']
    

    
    for j in range(1,len(sortedoccur)):
        
        sublight=light.loc[(light['Filter'] == sortedoccur[j][0]) 
                                     & (np.log10(lightresc['Time(s)']) <= 5) & (np.log10(lightresc['Time(s)']) > 4)].to_dict('subset')
        subx=np.log10(sublight['Time(s)'].values)
        suby=sublight['Mag'].values
        suberror_y=sublight['MagErr'].values
        
        
        indexeslist = [[p1,p2] for p1 in range(len(mostcommonx4)) for p2 in range(len(subx))]
    
        timediff = [np.abs(mostcommonx4[p1]-subx[p2]) for p1 in range(len(mostcommonx4)) for p2 in range(len(subx))]
        
        if len(timediff)==0:
            continue
        
        else:
            m = min(timediff)
            minpos = timediff.index(m)

            # We here define the re-scaling constant between the most common band and the j-th band (in order of occurencies)
            scalingfactor4=mostcommony4[indexeslist[minpos][0]]-suby[indexeslist[minpos][1]]
            scalingfactorslist[j][2].append(scalingfactor4)
    

    # 5th part of LC: 5<logt<6 => logdeltat<=0.15
    logdtprecision=0.15
    lightresc5=lightresc.loc[(np.log10(lightresc['Time(s)']) <= 6) & (np.log10(lightresc['Time(s)']) > 5)].to_dict('lightresc1')
    rescx5=np.log10(lightresc5['Time(s)'])
    rescy5=lightresc5['Mag']
    resccolor5=lightresc5['Filter']
    rescerror_y5= lightresc5['MagErr']
    reschover_data5=lightresc5['Source']    
    
    mostcommonlight5=light.loc[(light['Filter'] == mostcommonfilter) & (np.log10(lightresc['Time(s)']) <= 6) & (np.log10(lightresc['Time(s)']) > 5)].to_dict('list')
    
    
    mostcommonx5=np.log10(mostcommonlight5['Time(s)'])
    mostcommony5=mostcommonlight5['Mag']
    mostcommonerror_y5= mostcommonlight5['MagErr']
    
    
    # **************************** WORK IN PROGRESS ********************************************
    
    for j in range(1,len(sortedoccur)):
        
        sublight=light.loc[(light['Filter'] == sortedoccur[j][0]) 
                                     & (np.log10(lightresc['Time(s)']) <= 6) & (np.log10(lightresc['Time(s)']) > 5)].to_dict('subset')
        subx=np.log10(sublight['Time(s)'].values)
        suby=sublight['Mag'].values
        suberror_y=sublight['MagErr'].values

        
        indexeslist = [[p1,p2] for p1 in range(len(mostcommonx5)) for p2 in range(len(subx))]
    
        timediff = [np.abs(mostcommonx5[p1]-subx[p2]) for p1 in range(len(mostcommonx5)) for p2 in range(len(subx))]
        
        if len(timediff)==0:
            continue
        
        else:
            m = min(timediff)
            minpos = timediff.index(m)

            # We here define the re-scaling constant between the most common band and the j-th band (in order of occurencies)
            scalingfactor5=mostcommony5[indexeslist[minpos][0]]-suby[indexeslist[minpos][1]]
            scalingfactorslist[j][2].append(scalingfactor5)
            
    #print('Part5')     


    # 6th part of LC: logt>6 => logdeltat<=0.1
    logdtprecision=0.1
    lightresc6=lightresc.loc[(np.log10(lightresc['Time(s)']) > 6)].to_dict('lightresc1')
    rescx6=np.log10(lightresc6['Time(s)'])
    rescy6=lightresc6['Mag']
    resccolor6=lightresc6['Filter']
    rescerror_y6= lightresc6['MagErr']
    reschover_data6=lightresc6['Source']    
    
    mostcommonlight6=light.loc[(light['Filter'] == mostcommonfilter) & (np.log10(lightresc['Time(s)']) > 6)].to_dict('list')
    
    #print(mostcommonlight1)
    
    mostcommonx6=np.log10(mostcommonlight6['Time(s)'])
    mostcommony6=mostcommonlight6['Mag']
    mostcommonerror_y6=mostcommonlight6['MagErr']
    
    
    # **************************** WORK IN PROGRESS ********************************************
    
    for j in range(1,len(sortedoccur)):
        
        sublight=light.loc[(light['Filter'] == sortedoccur[j][0]) 
                                     & (np.log10(lightresc['Time(s)']) > 6)].to_dict('subset')
        subx=np.log10(sublight['Time(s)'].values)
        suby=sublight['Mag'].values
        suberror_y=sublight['MagErr'].values
        
        #scalingfactorslist.append([sortedoccur[j][0],sortedoccur[j][1],[]])
        
        indexeslist = [[p1,p2] for p1 in range(len(mostcommonx6)) for p2 in range(len(subx))]
        #print(indexeslist)
    
        timediff = [np.abs(mostcommonx6[p1]-subx[p2]) for p1 in range(len(mostcommonx6)) for p2 in range(len(subx))]
        
        if len(timediff)==0:
            continue
        
        else:
            m = min(timediff)
            minpos = timediff.index(m)

            # We here define the re-scaling constant between the most common band and the j-th band (in order of occurencies)
            scalingfactor6=mostcommony6[indexeslist[minpos][0]]-suby[indexeslist[minpos][1]]
            scalingfactorslist[j][2].append(scalingfactor6)
            
    
    # Averaging the rescaling factors for each filter
    
    averagedrescalingfactor=[]
    for j in range(len(scalingfactorslist)):
        if np.isnan(np.mean(scalingfactorslist[j][2]))==False:
                    averagedrescalingfactor.append([scalingfactorslist[j][0],scalingfactorslist[j][1],np.mean(scalingfactorslist[j][2])])
        if np.isnan(np.mean(scalingfactorslist[j][2]))==True:
                    averagedrescalingfactor.append([scalingfactorslist[j][0],scalingfactorslist[j][1],0])
    
    
    rescfactfromduplicate=[]
    
    for j in range(1,len(sortedoccur)):
        
        sublight=light.loc[light['Filter'] == sortedoccur[j][0]].to_dict('subset')
        subx=np.log10(sublight['Time(s)'].values)
        suby=sublight['Mag'].values
        suberror_y=sublight['MagErr'].values
        subcolor=sublight['Filter'].values
                    
        timediffpreliminar = []
        
        for p1 in range(len(mostcommonx)):
            for p2 in range(len(subx)):
                if (np.abs(mostcommonx[p1]-subx[p2])<=1e-15) & (subcolor[p2]==sortedoccur[j][0]):
                    timediffpreliminar.append([p1,p2,sortedoccur[j][0],mostcommony[p1]-suby[p2]])
        
        if len(timediffpreliminar)==0:
            continue
        
        
        meanlist=[]
        for k in timediffpreliminar:
            meanlist.append(k[3])
        
        rescfactfromduplicate.append([sortedoccur[j][0],np.mean(meanlist)])
    
    
    for a in averagedrescalingfactor:
        for b in rescfactfromduplicate:
            if a[0]==b[0]:
                a[2]=b[1]
                
    
    junction2to3=[]
    junction3to4=[]
    junction4to5=[]
    junction5to6=[]
    junction6up=[]
    
    nonrescaled=[]
    

    # RESCALING PART OF THE LC 0<logt<2
    
    for pp in range(len(rescy1)):
        logdtprecision=0.35
        start2=rescy1[pp]-rescerror_y1[pp]
        end2=rescy1[pp]+rescerror_y1[pp]
        supportlistov=[]
        supportlistdist=[]
        
        
        
        for ff in averagedrescalingfactor:
            if resccolor1[pp] == ff[0]:
                for nn in [x for x in range(len(rescy1)) if (x != pp) & (resccolor1[x]==mostcommonfilter)]:
                    start1=rescy1[nn]-rescerror_y1[nn] 
                    end1=rescy1[nn]+rescerror_y1[nn]
                    dist=np.abs(rescx1[nn]-rescx1[pp])
                    supportlistov.append(overlap(start1, end1, start2, end2))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    rescy1[pp]=rescy1[pp]
                    
                    if 2-logdtprecision<rescx1[pp]<2:
                        junction2to3.append([rescx1[pp],rescy1[pp],rescerror_y1[pp],resccolor1[pp],reschover_data1[pp]])                     
                    
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    rescy1[pp]=rescy1[pp]+ff[2]
                else:
                    rescy1[pp]=rescy1[pp]
                    nonrescaled.append([rescx1[pp],rescy1[pp],rescerror_y1[pp],resccolor1[pp],reschover_data1[pp]])
                   
    # RESCALING PART OF THE LC 2<logt<3                 
    
    for pp in range(len(rescy2)):
        start2=rescy2[pp]-rescerror_y2[pp]
        end2=rescy2[pp]+rescerror_y2[pp]
        supportlistov=[]
        supportlistdist=[]
        logdtprecision=0.3
        #junction3to4=[]
         
        
        for ff in averagedrescalingfactor:
            if resccolor2[pp] == ff[0]:
                for nn in [x for x in range(len(rescy2)) if (x != pp) & (resccolor2[x]==mostcommonfilter)]:
                    start1=rescy2[nn]-rescerror_y2[nn] 
                    end1=rescy2[nn]+rescerror_y2[nn]
                    dist=np.abs(rescx2[nn]-rescx2[pp])
                    supportlistov.append(overlap(start1, end1, start2, end2))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    rescy2[pp]=rescy2[pp]
                    
                    if 3-logdtprecision<rescx2[pp]<3:
                        junction3to4.append([rescx2[pp],rescy2[pp],rescerror_y2[pp],resccolor2[pp],reschover_data2[pp]]) 
                        
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    rescy2[pp]=rescy2[pp]+ff[2]
                else:
                    rescy2[pp]=rescy2[pp]
                    nonrescaled.append([rescx2[pp],rescy2[pp],rescerror_y2[pp],resccolor2[pp],reschover_data2[pp]])
                    
    # RESCALING PART OF THE LC 3<logt<4     
    
    for pp in range(len(rescy3)):
        
        start3=rescy3[pp]-rescerror_y3[pp]
        end3=rescy3[pp]+rescerror_y3[pp]
        supportlistov=[]
        supportlistdist=[]
        logdtprecision=0.25
        #junction4to5=[]
        
        
        for ff in averagedrescalingfactor:
            if resccolor3[pp] == ff[0]:
                for nn in [x for x in range(len(rescy3)) if (x != pp) & (resccolor3[x]==mostcommonfilter)]:
                    start1=rescy3[nn]-rescerror_y3[nn] 
                    end1=rescy3[nn]+rescerror_y3[nn]
                    dist=np.abs(rescx3[nn]-rescx3[pp])
                    supportlistov.append(overlap(start1, end1, start3, end3))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    rescy3[pp]=rescy3[pp]
                    
                    if 4-logdtprecision<rescx3[pp]<4:
                        junction4to5.append([rescx3[pp],rescy3[pp],rescerror_y3[pp],resccolor3[pp],reschover_data3[pp]])                    
                    
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    rescy3[pp]=rescy3[pp]+ff[2]
                else:
                    rescy3[pp]=rescy3[pp]
                    nonrescaled.append([rescx3[pp],rescy3[pp],rescerror_y3[pp],resccolor3[pp],reschover_data3[pp]])
      
                    
    # RESCALING PART OF THE LC 4<logt<5     
    
    for pp in range(len(rescy4)):
        
        start4=rescy4[pp]-rescerror_y4[pp]
        end4=rescy4[pp]+rescerror_y4[pp]
        supportlistov=[]
        supportlistdist=[]
        logdtprecision=0.2
        #junction5to6=[]
        
         
        
        for ff in averagedrescalingfactor:
            if resccolor4[pp] == ff[0]:
                for nn in [x for x in range(len(rescy4)) if (x != pp) & (resccolor4[x]==mostcommonfilter)]:
                    start1=rescy4[nn]-rescerror_y4[nn] 
                    end1=rescy4[nn]+rescerror_y4[nn]
                    dist=np.abs(rescx4[nn]-rescx4[pp])
                    supportlistov.append(overlap(start1, end1, start4, end4))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    rescy4[pp]=rescy4[pp]
                    
                    if 5-logdtprecision<rescx4[pp]<5:
                        junction5to6.append([rescx4[pp],rescy4[pp],rescerror_y4[pp],resccolor4[pp],reschover_data4[pp]])
                    
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    rescy4[pp]=rescy4[pp]+ff[2]
                else:
                    rescy4[pp]=rescy4[pp]
                    nonrescaled.append([rescx4[pp],rescy4[pp],rescerror_y4[pp],resccolor4[pp],reschover_data4[pp]])
                    
                    
    # RESCALING PART OF THE LC 5<logt<6 
    
    for pp in range(len(rescy5)):
        
        start5=rescy5[pp]-rescerror_y5[pp]
        end5=rescy5[pp]+rescerror_y5[pp]
        supportlistov=[]
        supportlistdist=[]
        logdtprecision=0.15
        #junction6up=[]
        
         
        
        for ff in averagedrescalingfactor:
            if resccolor5[pp] == ff[0]:
                for nn in [x for x in range(len(rescy5)) if (x != pp) & (resccolor5[x]==mostcommonfilter)]:
                    start1=rescy5[nn]-rescerror_y5[nn] 
                    end1=rescy5[nn]+rescerror_y5[nn]
                    dist=np.abs(rescx5[nn]-rescx5[pp])
                    supportlistov.append(overlap(start1, end1, start5, end5))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    rescy5[pp]=rescy5[pp]
                    
                    if 6-logdtprecision<rescx5[pp]<6:
                        junction6up.append([rescx5[pp],rescy5[pp],rescerror_y5[pp],resccolor5[pp],reschover_data5[pp]])
                        
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    rescy5[pp]=rescy5[pp]+ff[2]
                else:
                    rescy5[pp]=rescy5[pp]
                    nonrescaled.append([rescx5[pp],rescy5[pp],rescerror_y5[pp],resccolor5[pp],reschover_data5[pp]])
                    
                    
    # RESCALING PART OF THE LC Logt>6                    
    
                
    for pp in range(len(rescy6)):
        
        start6=rescy6[pp]-rescerror_y6[pp]
        end6=rescy6[pp]+rescerror_y6[pp]
        supportlistov=[]
        supportlistdist=[]
        logdtprecision=0.1
        
         
        
        for ff in averagedrescalingfactor:
            if resccolor6[pp] == ff[0]:
                for nn in [x for x in range(len(rescy6)) if (x != pp) & (resccolor6[x]==mostcommonfilter)]:
                    start1=rescy6[nn]-rescerror_y6[nn] 
                    end1=rescy6[nn]+rescerror_y6[nn]
                    dist=np.abs(rescx6[nn]-rescx6[pp])
                    supportlistov.append(overlap(start1, end1, start6, end6))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    rescy6[pp]=rescy6[pp]
                    
                    if 6-logdtprecision<rescx6[pp]<6:
                        junction6up.append([rescx6[pp],rescy6[pp],rescerror_y6[pp],resccolor6[pp],reschover_data6[pp]])                    
                    
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    rescy6[pp]=rescy6[pp]+ff[2]
                else:
                    rescy6[pp]=rescy6[pp]
                    nonrescaled.append([rescx6[pp],rescy6[pp],rescerror_y6[pp],resccolor6[pp],reschover_data6[pp]])
                    
                    
    #########################################################################
    # RESCALING THE JUNCTIONS                
    
    
    if len(junction2to3)!=0:
        for pp in range(len(junction2to3)):
        
            start3=junction2to3[pp][1]-junction2to3[pp][2]
            end3=junction2to3[pp][1]+junction2to3[pp][2]
            supportlistov=[]
            supportlistdist=[]
            logdtprecision=0.3
           
        
        for ff in averagedrescalingfactor:
            if junction2to3[pp][3] == ff[0]:
                for nn in [x for x in range(len(rescy3)) if (x != pp) & (resccolor3[x]==mostcommonfilter)]:
                    start1=rescy3[nn]-rescerror_y3[nn] 
                    end1=rescy3[nn]+rescerror_y3[nn]
                    dist=np.abs(rescx3[nn]-junction2to3[pp][0])
                    supportlistov.append(overlap(start1, end1, start3, end3))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    junction2to3[pp][1]=junction2to3[pp][1]
                    nonrescaled.append([junction2to3[pp][0],junction2to3[pp][1],junction2to3[pp][2],junction2to3[pp][3],junction2to3[pp][4]])
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    junction2to3[pp][1]=junction2to3[pp][1]+ff[2]
                else:
                    junction2to3[pp][1]=junction2to3[pp][1]
                    nonrescaled.append([junction2to3[pp][0],junction2to3[pp][1],junction2to3[pp][2],junction2to3[pp][3],junction2to3[pp][4]])
    else:
        True                      
    
    if len(junction3to4)!=0:    
        for pp in range(len(junction3to4)):
        
            start4=junction3to4[pp][1]-junction3to4[pp][2]
            end4=junction3to4[pp][1]+junction3to4[pp][2]
            supportlistov=[]
            supportlistdist=[]
            logdtprecision=0.25
        
         
        
        for ff in averagedrescalingfactor:
            if junction3to4[pp][3] == ff[0]:
                for nn in [x for x in range(len(rescy4)) if (x != pp) & (resccolor4[x]==mostcommonfilter)]:
                    start1=rescy4[nn]-rescerror_y4[nn] 
                    end1=rescy4[nn]+rescerror_y4[nn]
                    dist=np.abs(rescx4[nn]-junction3to4[pp][0])
                    supportlistov.append(overlap(start1, end1, start4, end4))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    junction3to4[pp][1]=junction3to4[pp][1]
                    nonrescaled.append([junction3to4[pp][0],junction3to4[pp][1],junction3to4[pp][2],junction3to4[pp][3],junction3to4[pp][4]])
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    junction3to4[pp][1]=junction3to4[pp][1]+ff[2]
                else:
                    junction3to4[pp][1]=junction3to4[pp][1]
                    nonrescaled.append([junction3to4[pp][0],junction3to4[pp][1],junction3to4[pp][2],junction3to4[pp][3],junction3to4[pp][4]])

    else:
        True                    
                    
        
    if len(junction4to5)!=0:    
        for pp in range(len(junction4to5)):
        
            start5=junction4to5[pp][1]-junction4to5[pp][2]
            end5=junction4to5[pp][1]+junction4to5[pp][2]
            supportlistov=[]
            supportlistdist=[]
            logdtprecision=0.2
        
         
        
        for ff in averagedrescalingfactor:
            if junction4to5[pp][3] == ff[0]:
                for nn in [x for x in range(len(rescy5)) if (x != pp) & (resccolor5[x]==mostcommonfilter)]:
                    start1=rescy5[nn]-rescerror_y5[nn] 
                    end1=rescy5[nn]+rescerror_y5[nn]
                    dist=np.abs(rescx5[nn]-junction4to5[pp][0])
                    supportlistov.append(overlap(start1, end1, start5, end5))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    junction4to5[pp][1]=junction4to5[pp][1]
                    nonrescaled.append([junction4to5[pp][0],junction4to5[pp][1],junction4to5[pp][2],junction4to5[pp][3],junction4to5[pp][4]])
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    junction4to5[pp][1]=junction4to5[pp][1]+ff[2]
                else:
                    junction4to5[pp][1]=junction4to5[pp][1]
                    nonrescaled.append([junction4to5[pp][0],junction4to5[pp][1],junction4to5[pp][2],junction4to5[pp][3],junction4to5[pp][4]])
 
    else:
        True   
    
#     print('Junction 5 to 6')
#     print(junction5to6)
#     print('Length of junction')
#     print(len(junction5to6))
#     print('Rescaling in 5<logt<6')
#     print(rescx6)
    
    if len(junction5to6)!=0:            
        for pp in range(len(junction5to6)):
        
            start6=junction5to6[pp][1]-junction5to6[pp][2]
            end6=junction5to6[pp][1]+junction5to6[pp][2]
            supportlistov=[]
            supportlistdist=[]
            logdtprecision=0.15
        
         
        
        for ff in averagedrescalingfactor:
            if junction5to6[pp][3] == ff[0]:
                for nn in [x for x in range(len(rescy6)) if (x != pp) & (resccolor6[x]==mostcommonfilter)]:
                    start1=rescy6[nn]-rescerror_y6[nn] 
                    end1=rescy6[nn]+rescerror_y6[nn]
                    dist=np.abs(rescx6[nn]-junction5to6[pp][0])
                    supportlistov.append(overlap(start1, end1, start6, end6))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    junction5to6[pp][1]=junction5to6[pp][1]
                    nonrescaled.append([junction5to6[pp][0],junction5to6[pp][1],junction5to6[pp][2],junction5to6[pp][3],junction5to6[pp][4]])
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    junction5to6[pp][1]=junction5to6[pp][1]+ff[2]
                else:
                    junction5to6[pp][1]=junction5to6[pp][1]
                    nonrescaled.append([junction5to6[pp][0],junction5to6[pp][1],junction5to6[pp][2],junction5to6[pp][3],junction5to6[pp][4]])

    else:
        True                   
                    
                    
    if len(junction6up)!=0:            
        for pp in range(len(junction6up)):
        
            start7=junction6up[pp][1]-junction6up[pp][2]
            end7=junction6up[pp][1]+junction6up[pp][2]
            supportlistov=[]
            supportlistdist=[]
            logdtprecision=0.10
        
         
        
        for ff in averagedrescalingfactor:
            if junction6up[pp][3] == ff[0]:
                for nn in [x for x in range(len(rescy6)) if (x != pp) & (resccolor6[x]==mostcommonfilter)]:
                    start1=rescy6[nn]-rescerror_y6[nn] 
                    end1=rescy6[nn]+rescerror_y6[nn]
                    dist=np.abs(rescx6[nn]-junction6up[pp][0])
                    supportlistov.append(overlap(start1, end1, start7, end7))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    junction6up[pp][1]=junction6up[pp][1]
                    nonrescaled.append([junction6up[pp][0],junction6up[pp][1],junction6up[pp][2],junction6up[pp][3],junction6up[pp][4]])
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    junction6up[pp][1]=junction6up[pp][1]+ff[2]
                else:
                    junction6up[pp][1]=junction6up[pp][1]  
                    nonrescaled.append([junction6up[pp][0],junction6up[pp][1],junction6up[pp][2],junction6up[pp][3],junction6up[pp][4]])

    else:
        True
            
    totaljunction=junction2to3+junction3to4+junction4to5+junction5to6+junction6up
    
    rescyfinal=np.concatenate((rescy1,rescy2,rescy3,rescy4,rescy5,rescy6))
    
    print('The most numerous filter of this GRB: ',sortedoccur[0][0],', with', sortedoccur[0][1], 'occurrences')
    
    
    rescdata = pd.DataFrame(list(zip(rescx, rescyfinal, rescerror_y, resccolor, freqfinal, reschover_data)),
               columns =['Time(s)','Mag','MagErr','Filter','Freq','Source'])
    
    
    for qq in totaljunction:
        
        for ind in rescdata.index:
            if (qq[0]==rescdata['Time(s)'][ind]) & (qq[3]==rescdata['Filter'][ind]):
                rescdata['Mag'][ind]=qq[1]
    
    #print(nonrescaled)
    #print(' ')
    
    nonrescaledclean=[]
    for item in nonrescaled:
        if item[3]!=mostcommonfilter:
            nonrescaledclean.append(item)
        
    #print(nonrescalednew)
    
    # IN THIS SECOND RIDE; WE SELECT THE SECOND MOST NUMEROUS FILTER AS TEMPLATE FOR A SECOND RESCALING
    
    # We here create the sublist of data points that have the second most numerous filter
    
    mostcommonlight=rescdata.loc[rescdata['Filter'] == secondcommonfilter].to_dict('subset')
    mostcommonx=mostcommonlight['Time(s)'].values
    mostcommony=mostcommonlight['Mag'].values
    mostcommonerror_y= mostcommonlight['MagErr'].values
    
    
    # Second step: find the minimum time difference between the most common and the next filter in order of occurrences
    
    # Here we start to create a list that contains the rescaling factors for every filter after the most common
    # The structure is (filtername, number of points, list of rescaling factors (up to 6))
    scalingfactorslist=[[sortedoccur[1][0],sortedoccur[1][1],[0]]]
    
    # The following for loop makes j go through all the other filters (after the most common)
    
        
    #print(scalingfactorslist)
    
    
    lightresc=pd.DataFrame(nonrescaledclean, columns=['Time(s)','Mag','MagErr','Filter','Source'])
    #lightresc=rescdata[rescdata['Filter'] != mostcommonfilter]  
    rescx=lightresc['Time(s)'].values
    rescy=lightresc['Mag'].values
    resccolor=lightresc['Filter'].values
    rescerror_y= lightresc['MagErr'].values
    reschover_data=lightresc['Source'].values
    
    # 1st part of LC: 0<logt<2 => logdeltat<=0.35
    logdtprecision=0.35
    lightresc1=lightresc.loc[(lightresc['Time(s)'] <= 2) & (lightresc['Time(s)'] > 0)].to_dict('lightresc1')
    rescx1=lightresc1['Time(s)']
    rescy1=lightresc1['Mag']
    resccolor1=lightresc1['Filter']
    rescerror_y1= lightresc1['MagErr'] 
    reschover_data1=lightresc1['Source']    
    
    mostcommonlight1=lightresc.loc[(light['Filter'] == secondcommonfilter) & (lightresc['Time(s)'] <= 2) & (lightresc['Time(s)'] > 0)].to_dict('list')
    
    #print(mostcommonlight1)
    
    mostcommonx1=mostcommonlight1['Time(s)']
    mostcommony1=mostcommonlight1['Mag']
    mostcommonerror_y1= mostcommonlight1['MagErr']
    
        
    # **************************** WORK IN PROGRESS ********************************************


    for j in range(1,len(sortedoccur)):
        scalingfactorslist.append([sortedoccur[j][0],sortedoccur[j][1],[]])
                 

    for j in range(1,len(sortedoccur)):
        
        sublight=lightresc.loc[(lightresc['Filter'] == sortedoccur[j][0]) 
                                     & (lightresc['Time(s)'] <= 2) & (lightresc['Time(s)'] > 0)].to_dict('subset')
        subx=sublight['Time(s)'].values
        suby=sublight['Mag'].values
        suberror_y=sublight['MagErr'].values
        
        
        indexeslist = [[p1,p2] for p1 in range(len(mostcommonx1)) for p2 in range(len(subx))]
        #print(indexeslist)
    
        timediff = [np.abs(mostcommonx1[p1]-subx[p2]) for p1 in range(len(mostcommonx1)) for p2 in range(len(subx))]
        
        if len(timediff)==0:
            continue
        
        else:
            m = min(timediff)
            minpos = timediff.index(m)
    
            # We here define the re-scaling constant between the most common band and the j-th band (in order of occurencies)
            scalingfactor1=mostcommony1[indexeslist[minpos][0]]-suby[indexeslist[minpos][1]]
            scalingfactorslist[j][2].append(scalingfactor1)
        
    #print('Part1')         
    #print(scalingfactorslist)

    # 2nd part of LC: 2<logt<3 => logdeltat<=0.3
    logdtprecision=0.3
    lightresc2=lightresc.loc[(lightresc['Time(s)'] <= 3) & (lightresc['Time(s)'] > 2)].to_dict('lightresc1')
    rescx2=lightresc2['Time(s)']
    rescy2=lightresc2['Mag']
    resccolor2=lightresc2['Filter']
    rescerror_y2= lightresc2['MagErr']
    reschover_data2=lightresc2['Source']    
    
    mostcommonlight2=lightresc.loc[(light['Filter'] == secondcommonfilter) & (lightresc['Time(s)'] <= 3) & (lightresc['Time(s)'] > 2)].to_dict('list')
    
    #print(mostcommonlight1)
    
    mostcommonx2=mostcommonlight2['Time(s)']
    mostcommony2=mostcommonlight2['Mag']
    mostcommonerror_y2= mostcommonlight2['MagErr']
    
    
    for j in range(1,len(sortedoccur)):
        
        sublight=lightresc.loc[(lightresc['Filter'] == sortedoccur[j][0]) 
                                     & (lightresc['Time(s)'] <= 3) & (lightresc['Time(s)'] > 2)].to_dict('subset')
        subx=sublight['Time(s)'].values
        suby=sublight['Mag'].values
        suberror_y=sublight['MagErr'].values
        
        #scalingfactorslist.append([sortedoccur[j][0],sortedoccur[j][1],[]])
        
        indexeslist = [[p1,p2] for p1 in range(len(mostcommonx2)) for p2 in range(len(subx))]
        #print(indexeslist)
    
        timediff = [np.abs(mostcommonx2[p1]-subx[p2]) for p1 in range(len(mostcommonx2)) for p2 in range(len(subx))]
        
        if len(timediff)==0:
            continue
        
        
        else:
            m = min(timediff)
            minpos = timediff.index(m)
    
            # We here define the re-scaling constant between the most common band and the j-th band (in order of occurencies)
            scalingfactor2=mostcommony2[indexeslist[minpos][0]]-suby[indexeslist[minpos][1]]
            scalingfactorslist[j][2].append(scalingfactor2)
        
    #print('Part2') 
    #print(scalingfactorslist)

    # 3rd part of LC: 3<logt<4 => logdeltat<=0.25
    logdtprecision=0.25
    lightresc3=lightresc.loc[(lightresc['Time(s)'] <= 4) & (lightresc['Time(s)'] > 3)].to_dict('lightresc1')
    rescx3=lightresc3['Time(s)']
    rescy3=lightresc3['Mag']
    resccolor3=lightresc3['Filter']
    rescerror_y3= lightresc3['MagErr']
    reschover_data3=lightresc3['Source']    
    
    mostcommonlight3=lightresc.loc[(light['Filter'] == secondcommonfilter) & (lightresc['Time(s)'] <= 4) & (lightresc['Time(s)'] > 3)].to_dict('list')
    
    #print(mostcommonlight1)
    
    mostcommonx3=mostcommonlight3['Time(s)']
    mostcommony3=mostcommonlight3['Mag']
    mostcommonerror_y3= mostcommonlight3['MagErr']
    
    
    # **************************** WORK IN PROGRESS ********************************************
    
    for j in range(1,len(sortedoccur)):
        
        sublight=lightresc.loc[(lightresc['Filter'] == sortedoccur[j][0]) 
                                     & (lightresc['Time(s)'] <= 4) & (lightresc['Time(s)'] > 3)].to_dict('subset')
        subx=sublight['Time(s)'].values
        suby=sublight['Mag'].values
        suberror_y=sublight['MagErr'].values
        
        #scalingfactorslist.append([sortedoccur[j][0],sortedoccur[j][1],[]])
        
        indexeslist = [[p1,p2] for p1 in range(len(mostcommonx3)) for p2 in range(len(subx))]
        #print(indexeslist)
    
        timediff = [np.abs(mostcommonx3[p1]-subx[p2]) for p1 in range(len(mostcommonx3)) for p2 in range(len(subx))]
        
        if len(timediff)==0:
            continue
        
        else:
            m = min(timediff)
            minpos = timediff.index(m)
    
            # We here define the re-scaling constant between the most common band and the j-th band (in order of occurencies)
            scalingfactor3=mostcommony3[indexeslist[minpos][0]]-suby[indexeslist[minpos][1]]
            scalingfactorslist[j][2].append(scalingfactor3)
                 
                
    # 4th part of LC: 4<logt<5 => logdeltat<=0.2
    logdtprecision=0.2
    lightresc4=lightresc.loc[(lightresc['Time(s)'] <= 5) & (lightresc['Time(s)'] > 4)].to_dict('lightresc1')
    rescx4=lightresc4['Time(s)']
    rescy4=lightresc4['Mag']
    resccolor4=lightresc4['Filter']
    rescerror_y4= lightresc4['MagErr']
    reschover_data4=lightresc4['Source']    
    
    mostcommonlight4=lightresc.loc[(light['Filter'] == secondcommonfilter) & (lightresc['Time(s)'] <= 5) & (lightresc['Time(s)'] > 4)].to_dict('list')
    
    #print(mostcommonlight1)
    
    mostcommonx4=mostcommonlight4['Time(s)']
    mostcommony4=mostcommonlight4['Mag']
    mostcommonerror_y4= mostcommonlight4['MagErr']
    
    
    # **************************** WORK IN PROGRESS ********************************************
    
    for j in range(1,len(sortedoccur)):
        
        sublight=lightresc.loc[(lightresc['Filter'] == sortedoccur[j][0]) 
                                     & (lightresc['Time(s)'] <= 5) & (lightresc['Time(s)'] > 4)].to_dict('subset')
        subx=sublight['Time(s)'].values
        suby=sublight['Mag'].values
        suberror_y=sublight['MagErr'].values
        
        #scalingfactorslist.append([sortedoccur[j][0],sortedoccur[j][1],[]])
        
        indexeslist = [[p1,p2] for p1 in range(len(mostcommonx4)) for p2 in range(len(subx))]
        #print(indexeslist)
    
        timediff = [np.abs(mostcommonx4[p1]-subx[p2]) for p1 in range(len(mostcommonx4)) for p2 in range(len(subx))]
        
        if len(timediff)==0:
            continue
        
        else:
            m = min(timediff)
            minpos = timediff.index(m)

            # We here define the re-scaling constant between the most common band and the j-th band (in order of occurencies)
            scalingfactor4=mostcommony4[indexeslist[minpos][0]]-suby[indexeslist[minpos][1]]
            scalingfactorslist[j][2].append(scalingfactor4)
            
    #print('Part4') 
    

    # 5th part of LC: 5<logt<6 => logdeltat<=0.15
    logdtprecision=0.15
    lightresc5=lightresc.loc[(lightresc['Time(s)'] <= 6) & (lightresc['Time(s)'] > 5)].to_dict('lightresc1')
    rescx5=lightresc5['Time(s)']
    rescy5=lightresc5['Mag']
    resccolor5=lightresc5['Filter']
    rescerror_y5= lightresc5['MagErr']
    reschover_data5=lightresc5['Source']    
    
    mostcommonlight5=lightresc.loc[(light['Filter'] == secondcommonfilter) & (lightresc['Time(s)'] <= 6) & (lightresc['Time(s)'] > 5)].to_dict('list')
    
    #print(mostcommonlight1)
    
    mostcommonx5=mostcommonlight5['Time(s)']
    mostcommony5=mostcommonlight5['Mag']
    mostcommonerror_y5= mostcommonlight5['MagErr']
    
    
    # **************************** WORK IN PROGRESS ********************************************
    
    for j in range(1,len(sortedoccur)):
        
        sublight=lightresc.loc[(lightresc['Filter'] == sortedoccur[j][0]) 
                                     & (lightresc['Time(s)'] <= 6) & (lightresc['Time(s)'] > 5)].to_dict('subset')
        subx=sublight['Time(s)'].values
        suby=sublight['Mag'].values
        suberror_y=sublight['MagErr'].values
        
        #scalingfactorslist.append([sortedoccur[j][0],sortedoccur[j][1],[]])
        
        indexeslist = [[p1,p2] for p1 in range(len(mostcommonx5)) for p2 in range(len(subx))]
        #print(indexeslist)
    
        timediff = [np.abs(mostcommonx5[p1]-subx[p2]) for p1 in range(len(mostcommonx5)) for p2 in range(len(subx))]
        
        if len(timediff)==0:
            continue
        
        else:
            m = min(timediff)
            minpos = timediff.index(m)

            # We here define the re-scaling constant between the most common band and the j-th band (in order of occurencies)
            scalingfactor5=mostcommony5[indexeslist[minpos][0]]-suby[indexeslist[minpos][1]]
            scalingfactorslist[j][2].append(scalingfactor5)
            
    #print('Part5')     


    # 6th part of LC: logt>6 => logdeltat<=0.1
    logdtprecision=0.1
    lightresc6=lightresc.loc[(lightresc['Time(s)'] > 6)].to_dict('lightresc1')
    rescx6=lightresc6['Time(s)']
    rescy6=lightresc6['Mag']
    resccolor6=lightresc6['Filter']
    rescerror_y6= lightresc6['MagErr']
    reschover_data6=lightresc6['Source']    
    
    mostcommonlight6=lightresc.loc[(light['Filter'] == secondcommonfilter) & (lightresc['Time(s)'] > 6)].to_dict('list')
    
    #print(mostcommonlight1)
    
    mostcommonx6=mostcommonlight6['Time(s)']
    mostcommony6=mostcommonlight6['Mag']
    mostcommonerror_y6=mostcommonlight6['MagErr']
    
    
    # **************************** WORK IN PROGRESS ********************************************
    
    for j in range(1,len(sortedoccur)):
        
        sublight=lightresc.loc[(lightresc['Filter'] == sortedoccur[j][0]) 
                                     & (lightresc['Time(s)'] > 6)].to_dict('subset')
        subx=sublight['Time(s)'].values
        suby=sublight['Mag'].values
        suberror_y=sublight['MagErr'].values
        
        #scalingfactorslist.append([sortedoccur[j][0],sortedoccur[j][1],[]])
        
        indexeslist = [[p1,p2] for p1 in range(len(mostcommonx6)) for p2 in range(len(subx))]
        #print(indexeslist)
    
        timediff = [np.abs(mostcommonx6[p1]-subx[p2]) for p1 in range(len(mostcommonx6)) for p2 in range(len(subx))]
        
        if len(timediff)==0:
            continue
        
        else:
            m = min(timediff)
            minpos = timediff.index(m)

            # We here define the re-scaling constant between the most common band and the j-th band (in order of occurencies)
            scalingfactor6=mostcommony6[indexeslist[minpos][0]]-suby[indexeslist[minpos][1]]
            scalingfactorslist[j][2].append(scalingfactor6)
            
    #print('Part6')
    
    
    # Averaging the rescaling factors for each filter
    
    averagedrescalingfactor=[]
    for j in range(len(scalingfactorslist)):
        if np.isnan(np.mean(scalingfactorslist[j][2]))==False:
                    averagedrescalingfactor.append([scalingfactorslist[j][0],scalingfactorslist[j][1],np.mean(scalingfactorslist[j][2])])
        if np.isnan(np.mean(scalingfactorslist[j][2]))==True:
                    averagedrescalingfactor.append([scalingfactorslist[j][0],scalingfactorslist[j][1],0])
    
    #print(scalingfactorslist)
    
    rescfactfromduplicate=[]
    
    for j in range(1,len(sortedoccur)):
        
        sublight=lightresc.loc[lightresc['Filter'] == sortedoccur[j][0]].to_dict('subset')
        subx=sublight['Time(s)'].values
        suby=sublight['Mag'].values
        suberror_y=sublight['MagErr'].values
        subcolor=sublight['Filter'].values
                    
        timediffpreliminar = []
        
        for p1 in range(len(mostcommonx)):
            for p2 in range(len(subx)):
                if (np.abs(mostcommonx[p1]-subx[p2])<=1e-15) & (subcolor[p2]==sortedoccur[j][0]):
                    timediffpreliminar.append([p1,p2,sortedoccur[j][0],mostcommony[p1]-suby[p2]])
        
        if len(timediffpreliminar)==0:
            continue
        
        #print('All the duplicates with the given filter')
        #print(timediffpreliminar)
        
        meanlist=[]
        for k in timediffpreliminar:
            meanlist.append(k[3])
        
        rescfactfromduplicate.append([sortedoccur[j][0],np.mean(meanlist)])
        
        #print('Averaging the rescaling factors for the given filter')
        #print(rescfactband)
    
    
    for a in averagedrescalingfactor:
        for b in rescfactfromduplicate:
            if a[0]==b[0]:
                a[2]=b[1]
                
    
    junction2to3=[]
    junction3to4=[]
    junction4to5=[]
    junction5to6=[]
    junction6up=[]
    
    

    # RESCALING PART OF THE LC 0<logt<2
    
    for pp in range(len(rescy1)):
        logdtprecision=0.35
        start2=rescy1[pp]-rescerror_y1[pp]
        end2=rescy1[pp]+rescerror_y1[pp]
        supportlistov=[]
        supportlistdist=[]
        
        
        
        for ff in averagedrescalingfactor:
            if (resccolor1[pp] == ff[0]) & (resccolor1[pp] != mostcommonfilter):
                for nn in [x for x in range(len(rescy1)) if (x != pp) & (resccolor1[x]==secondcommonfilter)]:
                    start1=rescy1[nn]-rescerror_y1[nn] 
                    end1=rescy1[nn]+rescerror_y1[nn]
                    dist=np.abs(rescx1[nn]-rescx1[pp])
                    supportlistov.append(overlap(start1, end1, start2, end2))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    rescy1[pp]=rescy1[pp]
                    
                    if 2-logdtprecision<rescx1[pp]<2:
                        junction2to3.append([rescx1[pp],rescy1[pp],rescerror_y1[pp],resccolor1[pp]])                     
                    
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    rescy1[pp]=rescy1[pp]+ff[2]
                else:
                    rescy1[pp]=rescy1[pp]
    
    # RESCALING PART OF THE LC 2<logt<3                 
    
    for pp in range(len(rescy2)):
        start2=rescy2[pp]-rescerror_y2[pp]
        end2=rescy2[pp]+rescerror_y2[pp]
        supportlistov=[]
        supportlistdist=[]
        logdtprecision=0.3
        #junction3to4=[]
         
        
        for ff in averagedrescalingfactor:
            if (resccolor2[pp] == ff[0]) & (resccolor2[pp] != mostcommonfilter):
                for nn in [x for x in range(len(rescy2)) if (x != pp) & (resccolor2[x]==secondcommonfilter)]:
                    start1=rescy2[nn]-rescerror_y2[nn] 
                    end1=rescy2[nn]+rescerror_y2[nn]
                    dist=np.abs(rescx2[nn]-rescx2[pp])
                    supportlistov.append(overlap(start1, end1, start2, end2))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    rescy2[pp]=rescy2[pp]
                    
                    if 3-logdtprecision<rescx2[pp]<3:
                        junction3to4.append([rescx2[pp],rescy2[pp],rescerror_y2[pp],resccolor2[pp]]) 
                        
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    rescy2[pp]=rescy2[pp]+ff[2]
                else:
                    rescy2[pp]=rescy2[pp]
 

    # RESCALING PART OF THE LC 3<logt<4     
    
    for pp in range(len(rescy3)):
        
        start3=rescy3[pp]-rescerror_y3[pp]
        end3=rescy3[pp]+rescerror_y3[pp]
        supportlistov=[]
        supportlistdist=[]
        logdtprecision=0.25
        #junction4to5=[]
        
        
        for ff in averagedrescalingfactor:
            if (resccolor3[pp] == ff[0]) & (resccolor3[pp] != mostcommonfilter):
                for nn in [x for x in range(len(rescy3)) if (x != pp) & (resccolor3[x]==secondcommonfilter)]:
                    start1=rescy3[nn]-rescerror_y3[nn] 
                    end1=rescy3[nn]+rescerror_y3[nn]
                    dist=np.abs(rescx3[nn]-rescx3[pp])
                    supportlistov.append(overlap(start1, end1, start3, end3))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    rescy3[pp]=rescy3[pp]
                    
                    if 4-logdtprecision<rescx3[pp]<4:
                        junction4to5.append([rescx3[pp],rescy3[pp],rescerror_y3[pp],resccolor3[pp]])                    
                    
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    rescy3[pp]=rescy3[pp]+ff[2]
                else:
                    rescy3[pp]=rescy3[pp]

                    
    # RESCALING PART OF THE LC 4<logt<5     
    
    for pp in range(len(rescy4)):
        
        start4=rescy4[pp]-rescerror_y4[pp]
        end4=rescy4[pp]+rescerror_y4[pp]
        supportlistov=[]
        supportlistdist=[]
        logdtprecision=0.2
        #junction5to6=[]
        
         
        
        for ff in averagedrescalingfactor:
            if (resccolor4[pp] == ff[0]) & (resccolor4[pp] != mostcommonfilter):
                for nn in [x for x in range(len(rescy4)) if (x != pp) & (resccolor4[x]==secondcommonfilter)]:
                    start1=rescy4[nn]-rescerror_y4[nn] 
                    end1=rescy4[nn]+rescerror_y4[nn]
                    dist=np.abs(rescx4[nn]-rescx4[pp])
                    supportlistov.append(overlap(start1, end1, start4, end4))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    rescy4[pp]=rescy4[pp]
                    
                    if 5-logdtprecision<rescx4[pp]<5:
                        junction5to6.append([rescx4[pp],rescy4[pp],rescerror_y4[pp],resccolor4[pp]])
                    
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    rescy4[pp]=rescy4[pp]+ff[2]
                else:
                    rescy4[pp]=rescy4[pp]

                    
    # RESCALING PART OF THE LC 5<logt<6 
    
    for pp in range(len(rescy5)):
        
        start5=rescy5[pp]-rescerror_y5[pp]
        end5=rescy5[pp]+rescerror_y5[pp]
        supportlistov=[]
        supportlistdist=[]
        logdtprecision=0.15
        #junction6up=[]
        
         
        
        for ff in averagedrescalingfactor:
            if (resccolor5[pp] == ff[0]) & (resccolor5[pp] != mostcommonfilter):
                for nn in [x for x in range(len(rescy5)) if (x != pp) & (resccolor5[x]==secondcommonfilter)]:
                    start1=rescy5[nn]-rescerror_y5[nn] 
                    end1=rescy5[nn]+rescerror_y5[nn]
                    dist=np.abs(rescx5[nn]-rescx5[pp])
                    supportlistov.append(overlap(start1, end1, start5, end5))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    rescy5[pp]=rescy5[pp]
                    
                    if 6-logdtprecision<rescx5[pp]<6:
                        junction6up.append([rescx5[pp],rescy5[pp],rescerror_y5[pp],resccolor5[pp]])
                        
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    rescy5[pp]=rescy5[pp]+ff[2]
                else:
                    rescy5[pp]=rescy5[pp]
                    
              
    # RESCALING PART OF THE LC Logt>6                    
    
                
    for pp in range(len(rescy6)):
        
        start6=rescy6[pp]-rescerror_y6[pp]
        end6=rescy6[pp]+rescerror_y6[pp]
        supportlistov=[]
        supportlistdist=[]
        logdtprecision=0.1
        
         
        
        for ff in averagedrescalingfactor:
            if (resccolor6[pp] == ff[0]) & (resccolor6[pp] != mostcommonfilter):
                for nn in [x for x in range(len(rescy6)) if (x != pp) & (resccolor6[x]==secondcommonfilter)]:
                    start1=rescy6[nn]-rescerror_y6[nn] 
                    end1=rescy6[nn]+rescerror_y6[nn]
                    dist=np.abs(rescx6[nn]-rescx6[pp])
                    supportlistov.append(overlap(start1, end1, start6, end6))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    rescy6[pp]=rescy6[pp]
                    
                    if 6-logdtprecision<rescx6[pp]<6:
                        junction6up.append([rescx6[pp],rescy6[pp],rescerror_y6[pp],resccolor6[pp]])                    
                    
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    rescy6[pp]=rescy6[pp]+ff[2]
                else:
                    rescy6[pp]=rescy6[pp]

    
    #########################################################################
    # RESCALING THE JUNCTIONS                
    
    
    if len(junction2to3)!=0:
        for pp in range(len(junction2to3)):
        
            start3=junction2to3[pp][1]-junction2to3[pp][2]
            end3=junction2to3[pp][1]+junction2to3[pp][2]
            supportlistov=[]
            supportlistdist=[]
            logdtprecision=0.3
           
        
        for ff in averagedrescalingfactor:
            if (junction2to3[pp][3] == ff[0]) & (junction2to3[pp][3] != mostcommonfilter):
                for nn in [x for x in range(len(rescy3)) if (x != pp) & (resccolor3[x]==mostcommonfilter)]:
                    start1=rescy3[nn]-rescerror_y3[nn] 
                    end1=rescy3[nn]+rescerror_y3[nn]
                    dist=np.abs(rescx3[nn]-junction2to3[pp][0])
                    supportlistov.append(overlap(start1, end1, start3, end3))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    junction2to3[pp][1]=junction2to3[pp][1]
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    junction2to3[pp][1]=junction2to3[pp][1]+ff[2]
                else:
                    junction2to3[pp][1]=junction2to3[pp][1]

    else:
        junction2to3=[]                      
    
    if len(junction3to4)!=0:    
        for pp in range(len(junction3to4)):
        
            start4=junction3to4[pp][1]-junction3to4[pp][2]
            end4=junction3to4[pp][1]+junction3to4[pp][2]
            supportlistov=[]
            supportlistdist=[]
            logdtprecision=0.25
        
         
        
        for ff in averagedrescalingfactor:
            if (junction3to4[pp][3] == ff[0]) & (junction3to4[pp][3] != secondcommonfilter):
                for nn in [x for x in range(len(rescy4)) if (x != pp) & (resccolor4[x]==mostcommonfilter)]:
                    start1=rescy4[nn]-rescerror_y4[nn] 
                    end1=rescy4[nn]+rescerror_y4[nn]
                    dist=np.abs(rescx4[nn]-junction3to4[pp][0])
                    supportlistov.append(overlap(start1, end1, start4, end4))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    junction3to4[pp][1]=junction3to4[pp][1]
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    junction3to4[pp][1]=junction3to4[pp][1]+ff[2]
                else:
                    junction3to4[pp][1]=junction3to4[pp][1]
                    
    else:
        junction3to4=[]                    
                    
        
    if len(junction4to5)!=0:    
        for pp in range(len(junction4to5)):
        
            start5=junction4to5[pp][1]-junction4to5[pp][2]
            end5=junction4to5[pp][1]+junction4to5[pp][2]
            supportlistov=[]
            supportlistdist=[]
            logdtprecision=0.2
        
         
        
        for ff in averagedrescalingfactor:
            if (junction4to5[pp][3] == ff[0]) & (junction4to5[pp][3] != secondcommonfilter):
                for nn in [x for x in range(len(rescy5)) if (x != pp) & (resccolor5[x]==mostcommonfilter)]:
                    start1=rescy5[nn]-rescerror_y5[nn] 
                    end1=rescy5[nn]+rescerror_y5[nn]
                    dist=np.abs(rescx5[nn]-junction4to5[pp][0])
                    supportlistov.append(overlap(start1, end1, start5, end5))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    junction4to5[pp][1]=junction4to5[pp][1]
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    junction4to5[pp][1]=junction4to5[pp][1]+ff[2]
                else:
                    junction4to5[pp][1]=junction4to5[pp][1]
 
    else:
        junction4to5=[]   

        
    if len(junction5to6)!=0:            
        for pp in range(len(junction5to6)):
        
            start6=junction5to6[pp][1]-junction5to6[pp][2]
            end6=junction5to6[pp][1]+junction5to6[pp][2]
            supportlistov=[]
            supportlistdist=[]
            logdtprecision=0.15
        
         
        
        for ff in averagedrescalingfactor:
            if (junction5to6[pp][3] == ff[0]) & (junction5to6[pp][3] != mostcommonfilter):
                for nn in [x for x in range(len(rescy6)) if (x != pp) & (resccolor6[x]==mostcommonfilter)]:
                    start1=rescy6[nn]-rescerror_y6[nn] 
                    end1=rescy6[nn]+rescerror_y6[nn]
                    dist=np.abs(rescx6[nn]-junction5to6[pp][0])
                    supportlistov.append(overlap(start1, end1, start6, end6))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    junction5to6[pp][1]=junction5to6[pp][1]
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    junction5to6[pp][1]=junction5to6[pp][1]+ff[2]
                else:
                    junction5to6[pp][1]=junction5to6[pp][1]

    else:
        junction5to6=[]                   
                    
                    
    if len(junction6up)!=0:            
        for pp in range(len(junction6up)):
        
            start7=junction6up[pp][1]-junction6up[pp][2]
            end7=junction6up[pp][1]+junction6up[pp][2]
            supportlistov=[]
            supportlistdist=[]
            logdtprecision=0.10
        
         
        
        for ff in averagedrescalingfactor:
            if (junction6up[pp][3] == ff[0]) & (junction6up[pp][3] != mostcommonfilter):
                for nn in [x for x in range(len(rescy6)) if (x != pp) & (resccolor6[x]==mostcommonfilter)]:
                    start1=rescy6[nn]-rescerror_y6[nn] 
                    end1=rescy6[nn]+rescerror_y6[nn]
                    dist=np.abs(rescx6[nn]-junction6up[pp][0])
                    supportlistov.append(overlap(start1, end1, start7, end7))
                    supportlistdist.append(dist)
                          
                if all(supportlistdist)>logdtprecision:
                    junction6up[pp][1]=junction6up[pp][1]
                if any((supportlistov[i] == 0) & (supportlistdist[i] <= logdtprecision) for i in range(len(supportlistov))):
                    junction6up[pp][1]=junction6up[pp][1]+ff[2]
                else:
                    junction6up[pp][1]=junction6up[pp][1]

    else:
        junction6up=[]
            
    totaljunction2=junction2to3+junction3to4+junction4to5+junction5to6+junction6up
    
    rescyfinal2=np.concatenate((rescy1,rescy2,rescy3,rescy4,rescy5,rescy6))
    
    print('The second most numerous filter of this GRB: ',sortedoccur[1][0],', with', sortedoccur[1][1], 'occurrences')
    
    
    rescdata2 = pd.DataFrame(list(zip(rescx, rescyfinal2, rescerror_y, resccolor, freqfinal, reschover_data)),
               columns =['Time(s)','Mag','MagErr','Filter','Freq','Source'])
    
    
    for qq in totaljunction:
        
        for ind in rescdata2.index:
            if (qq[0]==rescdata2['Time(s)'][ind]) & (qq[3]==rescdata2['Filter'][ind]):
                rescdata2['Mag'][ind]=qq[1]
    
    
    # MERGING THE FIRST RESCALED LC (WITH RESPECT TO MOST NUMEROUS FILTER) AND THE SECOND RESCALED
    # WITH RESPECT TO SECOND MOST NUMEROUS FILTER
    
#     tosubstitute=pd.DataFrame()
#     for s in light.index:
#         for o in rescdata.index:
#             if (rescdata['Time(s)'][o]==light['Time(s)'][s]) & (rescdata['Mag'][o]==light['Mag'][s]) & (rescdata['MagErr'][o]==light['MagErr'][s]) & (rescdata['Filter'][o]==light['Filter'][s]):
#                 tosubstitute.append(rescdata[o])
    
                
    for o in rescdata.index:
        for j in rescdata2.index:
                if (rescdata2['Time(s)'][j]==rescdata['Time(s)'][o]) & (rescdata2['MagErr'][j]==rescdata['MagErr'][o]) & (rescdata2['Filter'][j]==rescdata['Filter'][o]):
                    rescdata['Mag'][o]=rescdata2['Mag'][j]
    
    
    # THIRD STEP
    # Duplicate remover written by Ridha. Keeps the most occurring filter. BIAGIO'S EDIT: I put the rounding on time values
    
    def duplicate_remover(df):
        freq_df = pd.DataFrame(df['Filter'].value_counts())
        freq_df.rename(columns = {'Filter':'freq'}, inplace = True)
    
        for i,j in zip(df.index, df.Filter):
            df.loc[i, 'freq'] = freq_df.loc[j, 'freq']

        #df = df.sort_values('freq', ascending=False).drop_duplicates(subset='Time(s)', keep="first")
        
        df = df.sort_values('freq', ascending=False)
        digitsnumber = 2
        df = df.round({'Time(s)': digitsnumber }) #rounding the values up to the digitsnumber parameter
        df = df.drop_duplicates(subset='Time(s)', keep="first")
        df = df.sort_values('Time(s)', ascending=True)

        return df
    
    rescdatafinal=duplicate_remover(rescdata)
    
    
    colorandfreq=[]
    for k in rescdatafinal['Filter']:
        for j in filterslist:
            if j[0]==k:
                colorandfreq.append(str(k)+' ('+str(j[1])+')')

    
    fig = px.scatter(
    data_frame=rescdatafinal,
    x=rescdatafinal['Time(s)'].values,
    y=-rescdatafinal['Mag'].values,#=np.log10(resclight['flux'].values),
    error_y=rescdatafinal['MagErr'].values,#= resclight['flux_err'].values / (light['flux'].values * np.log(10)),
    hover_data=['Source'],#=['source'],
    color=colorandfreq, #rescdatafinal['Filter'].values,
    title=i+'_RESCALED',
    )
    fig.update_layout(
        xaxis_title='log Time (s)',
        yaxis_title='Mag'
    )

    fig.show()
    #fig.write_html(i+"_RESCALED.html")